In [ ]:
suppressMessages(library(ArchR))
suppressMessages(library(patchwork))

### Set parameters

In [ ]:
set.seed(42)
addArchRThreads(threads = parallel::detectCores() - 2)
addArchRGenome("hg38")

### Create Arrow files

In [ ]:
## create Arrow files
inputFiles <- c("CK166" = "../../Alignment/CK166/outs/fragments.tsv.gz",
                "CK170" = "../../Alignment/CK170/outs/fragments.tsv.gz",
                "CK174" = "../../Alignment/CK174/outs/fragments.tsv.gz",
                "CK171" = "../../Alignment/CK171/outs/fragments.tsv.gz",
                "CK169" = "../../Alignment/CK169/outs/fragments.tsv.gz",
                "CK168" = "../../Alignment/CK168/outs/fragments.tsv.gz",
                "CK173" = "../../Alignment/CK173/outs/fragments.tsv.gz",
                "CK167" = "../../Alignment/CK167/outs/fragments.tsv.gz",
                "CK353" = "../../Alignment/CK353/outs/fragments.tsv.gz",
                "CK381" = "../../Alignment/CK381/outs/fragments.tsv.gz",
                "CK336" = "../../Alignment/CK336/outs/fragments.tsv.gz",
                "CK337" = "../../Alignment/CK337/outs/fragments.tsv.gz",
                "CK338" = "../../Alignment/CK338/outs/fragments.tsv.gz",
                "CK339" = "../../Alignment/CK339/outs/fragments.tsv.gz",
                "CK340" = "../../Alignment/CK340/outs/fragments.tsv.gz",
                "CK341" = "../../Alignment/CK341/outs/fragments.tsv.gz",
                "CK380" = "../../Alignment/CK380/outs/fragments.tsv.gz",
                "CK343" = "../../Alignment/CK343/outs/fragments.tsv.gz",
                "CK354" = "../../Alignment/CK354/outs/fragments.tsv.gz",
                "CK344" = "../../Alignment/CK344/outs/fragments.tsv.gz",
                "CK385" = "../../Alignment/CK385/outs/fragments.tsv.gz",
                "CK386" = "../../Alignment/CK386/outs/fragments.tsv.gz",
                "CK346" = "../../Alignment/CK346/outs/fragments.tsv.gz",
                "CK387" = "../../Alignment/CK387/outs/fragments.tsv.gz",
                "CK388" = "../../Alignment/CK388/outs/fragments.tsv.gz",
                "CK389" = "../../Alignment/CK389/outs/fragments.tsv.gz",
                "CK355" = "../../Alignment/CK355/outs/fragments.tsv.gz",
                "CK349" = "../../Alignment/CK349/outs/fragments.tsv.gz",
                "CK382" = "../../Alignment/CK382/outs/fragments.tsv.gz",
                "CK350" = "../../Alignment/CK350/outs/fragments.tsv.gz",
                "CK351" = "../../Alignment/CK351/outs/fragments.tsv.gz",
                "CK352" = "../../Alignment/CK352/outs/fragments.tsv.gz",
                "CK383" = "../../Alignment/CK383/outs/fragments.tsv.gz",
                "CK390" = "../../Alignment/CK390/outs/fragments.tsv.gz",
                "CK391" = "../../Alignment/CK391/outs/fragments.tsv.gz"
               )

minTSS <- 4
minFrags <- 3000

ArrowFiles <- createArrowFiles(
  inputFiles = inputFiles,
  sampleNames = names(inputFiles),
  outputNames = names(inputFiles),
  minTSS = minTSS, 
  minFrags = minFrags, 
  QCDir = "../data/QualityControl",
  addTileMat = TRUE,
  addGeneScoreMat = TRUE
)

### Visualize quality control

In [ ]:
## plot quality control
plotlist <- lapply(names(inputFiles), function(sample){
  input_filename <- sprintf("../data/QualityControl/%s/%s-Pre-Filter-Metadata.rds", sample, sample)
    
    if(file.exists(input_filename)){
        Metadata <- readRDS(input_filename)
        
        ggtitle <- sprintf("%s",
            paste0(sample, "\nnCells Pass Filter = ", sum(Metadata$Keep))
          )
        
        gg <- ggPoint(
          x = pmin(log10(Metadata$nFrags), 5) + rnorm(length(Metadata$nFrags), sd = 0.00001),
          y = Metadata$TSSEnrichment + rnorm(length(Metadata$nFrags), sd = 0.00001), 
          colorDensity = TRUE,
          xlim = c(2.5, 5),
          ylim = c(0, max(Metadata$TSSEnrichment) * 1.05),
          baseSize = 6,
          continuousSet = "sambaNight",
          xlabel = "Log 10 (Unique Fragments)",
          ylabel = "TSS Enrichment",
          title = ggtitle,
          rastr = TRUE) + 
          geom_hline(yintercept=minTSS, lty = "dashed", size = 0.25) +
          geom_vline(xintercept=log10(minFrags), lty = "dashed", size = 0.25) +
            theme(plot.margin = margin(0.1, 0.1, 0.1, 0.1, unit = "in"))
        
        return(gg)  
        }
})
               
p <- patchwork::wrap_plots(plotlist, ncol = 7)
                   
options(repr.plot.width = 28, repr.height = 16)
                   
print(p)

In [ ]:
## Creating an ArchRProject
proj <- ArchRProject(
  ArrowFiles = ArrowFiles, 
  outputDirectory = "../data/VisiumHeart",
  showLogo = FALSE,
  copyArrows = TRUE #This is recommened so that you maintain an unaltered copy for later usage.
)

getAvailableMatrices(proj)

In [ ]:
# Inferring scATAC-seq Doublets with ArchR
proj <- addDoubletScores(
    input = proj,
    k = 10, #Refers to how many cells near a "pseudo-doublet" to count.
    knnMethod = "UMAP", #Refers to the embedding to use for nearest neighbor search with doublet projection.
    LSIMethod = 1,
    outDir = "../data/DoubletScores"
)

In [ ]:
## remove doublets
proj <- filterDoublets(proj)

In [ ]:
## Save data
saveArchRProject(ArchRProj = proj, 
                 load = FALSE)

In [ ]:
## Session information

sessionInfo()